In [1]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the pre-trained model
model = keras.models.load_model(os.path.abspath(os.path.join("model", "LSTM_1_model_saved_model")))

# Load the test dataset
test = pd.read_csv(os.path.join("dataset", "test.csv"))

# Ensure the new data is preprocessed in the same way as the training data
scaler = MinMaxScaler()
X_new = test[['arrival_hour', 'arrival_minute', 'stop_lat', 'stop_lon', 'next_lat', 'next_lon', 'speed_kmh',
              'segment_max_speed_kmh', 'runtime_sec', 'direction_id', 'distance_m']]
X_new = scaler.fit_transform(X_new)

# Define the sequence length (seq_length) used during training
seq_length = 64

# Generate sequences from the new data
X_seq_new = []
num_objects = len(X_new)

for i in range(num_objects):
    start = max(0, i - seq_length + 1)
    sequence = X_new[start:i + 1]
    X_seq_new.append(sequence)

# Pad sequences to ensure consistent length
X_seq_new = pad_sequences(X_seq_new, maxlen=seq_length, dtype='float32', padding='post', truncating='post')

# Assuming 'arrival_time' is the column in the 'test' DataFrame representing the time steps
timestamps = test['arrival_time'].iloc[seq_length - 1:num_objects]

# Make predictions
predicted_congestion = model.predict(X_seq_new)

location = test[["stop_lat", "stop_lon", "next_lat", "next_lon", "arrival_hour", "arrival_minute"]]
location["predicted_congestion"] = predicted_congestion

C:\Users\vinhn\AppData\Local\Temp\ipykernel_30140\1454590558.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  location["predicted_congestion"] = predicted_congestion


In [2]:
location

,stop_lat,stop_lon,next_lat,next_lon,arrival_hour,arrival_minute,predicted_congestion
0,51.154290,7.066125,51.151736,7.055332,19,25,3.075770
1,51.151736,7.055332,51.150643,7.050905,19,26,3.487577
2,51.150643,7.050905,51.148841,7.045405,19,27,3.744970
3,51.148841,7.045405,51.146150,7.029510,19,28,3.328794
4,51.146150,7.029510,51.142660,7.027789,19,29,2.322833
...,...,...,...,...,...,...,...
95,51.167788,7.359327,51.173025,7.362712,5,9,0.534638
96,51.173025,7.362712,51.178622,7.362978,5,10,1.196557
97,51.178622,7.362978,51.185974,7.366155,5,11,2.852059
98,51.185974,7.366155,51.189468,7.373354,5,12,1.100611
